In [1]:
import numpy as np
import pandas as pd
import math
from collections import deque

In [6]:
current_oracle = 0
blocks = []
oracle_values =  deque([])

def create_mempool(txn_number, sq_df, post_df, t=0):
    global current_oracle

    values = np.random.pareto(1, txn_number)
    gas_limit = 10**5 * np.random.pareto(2, txn_number)
    sq_gas_price = [current_oracle * v for v in values]

    sq_df = sq_df.append(pd.DataFrame({
        'gas price': sq_gas_price,
        'gas limit': gas_limit,
        'time' : t,
        'fee cap': [x * y for x,y in zip(sq_gas_price,gas_limit)]
        }), ignore_index=True)

    post_df = post_df.append(pd.DataFrame({
        'gas price': None,
        'gas limit': gas_limit,
        'time': t
        }), ignore_index=True)
    
    sq_df = sq_df.sort_values(by=['gas price'], ascending=False).reset_index(drop=True)
    post_df = post_df.sort_values(by=['gas price'], ascending=False).reset_index(drop=True)
    

    
    return sq_df, post_df

# def get_demand():
#     # average: 50%, high: 25%, low: 25%
#     r = random.random()
#     if r < .5:
#         return 70
#     elif r < .75:
#         return 100
#     else:
#         return 50

#gets minimum gas price of last 100 blocks and returns 60th percentile value
def get_oracle():
    global current_oracle, oracle_values, blocks
    recent_block = blocks[-1]
    min_gp = math.inf
    for txn in recent_block:
        if(txn[1] < min_gp):
            min_gp = txn[1]
    
    oracle_values.popleft()
    oracle_values.append(min_gp)

    sorted_oracle_values = sorted(oracle_values)
  
    current_oracle = sorted_oracle_values[59]

def sq_fill_block(sq_df):
    block = []
    block_size = 0
    for i in range(len(sq_df)):
        txn = sq_df.iloc[i, :].to_list()
        if block_size + txn[1] > 15000000:
            break
        else:
            block += [txn]
            block_size += txn[1]
    
    sq_df = sq_df.iloc[i+1:,:]    
    
    return block, sq_df
        
def simulate(n):
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    block_data = pd.DataFrame()
    mempool_data = pd.DataFrame()
    
    global oracle_values, blocks, current_oracle
    
    data = pd.read_csv('block_data.csv')
    minGasdf = data[['minGasPrice']].values
    for d in minGasdf:
        if len(oracle_values) == 100:
            break
        if d == 'None':
            continue
        oracle_values.append(int(d[0]))
  
    sorted_oracle_values = sorted(oracle_values)
    current_oracle = sorted_oracle_values[59]    
    
    sq_data, post_data = create_mempool(2000, df1, df2)

    for i in range(n):
        filled_block = sq_fill_block(sq_data)
        blocks += [filled_block[0]]
        get_oracle()
        
        block_data = block_data.append(pd.DataFrame({
            'number of txns': [len(filled_block[0])],
            'total gas': [sum([filled_block[0][i][1] for i in range(len(filled_block[0]))])],
            'average gas price': [sum([filled_block[0][i][1] for i in range(len(filled_block[0]))])/len(filled_block[0])]
        }), ignore_index=True)
        
        sq_data = filled_block[1]
        mempool_data = mempool_data.append(pd.DataFrame({
            "average gas price": [sq_data.sum()[0]/len(sq_data)],
            "average weighted value": [sq_data.sum()[3]/len(sq_data)]
        }), ignore_index=True)
        if i+1 == n:
            break
        if sq_data.shape[0] == 0:
          print('SQ mempool is empty')
        sq_data, post_data = create_mempool(700, sq_data, post_data,i+1)
        
    return blocks, sq_data, block_data, mempool_data
  


In [7]:
s = simulate(50)
print("Mempool summary data:\n", s[3])
print("SQ mempool data:\n", s[1])
print("Block summary data:\n", s[2])

Mempool summary data:
     average gas price  average weighted value
0        9.187262e+10            8.877211e+15
1        6.054815e+10            6.163164e+15
2        5.310393e+10            5.516780e+15
3        4.846529e+10            4.973858e+15
4        4.516771e+10            4.802543e+15
5        4.770368e+10            4.972535e+15
6        4.591192e+10            4.559954e+15
7        4.557081e+10            4.525060e+15
8        4.442145e+10            4.398782e+15
9        4.431692e+10            4.366628e+15
10       4.229332e+10            4.163350e+15
11       4.167441e+10            4.115886e+15
12       4.175574e+10            4.124399e+15
13       4.143069e+10            4.148506e+15
14       4.207893e+10            4.204036e+15
15       4.114191e+10            4.100677e+15
16       4.030061e+10            4.054853e+15
17       3.960864e+10            3.967123e+15
18       3.965606e+10            3.937029e+15
19       3.960134e+10            3.936275e+15
20       3.